# chapter 

## Setting
- GPT 모델 설정 필요

In [1]:
import os
key_file = open('key.txt', 'r')
os.environ['OPENAI_API_KEY'] = key_file.readline()
key_file.close()
from openai import OpenAI
client = OpenAI()
model = "o4-mini"

In [2]:
%%time

messages = []

system_content = '''
# 게임 설명
메이즈 게임은 미로 찾기 게임이고, 파이썬으로 아바타가 도착점으로 가도록 만들어야 하는 게임이다.
'''

user_content = '''
'''

messages.append({"role": "system", "content": system_content})
messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)

******************** answer ********************
안녕하세요! 메이즈 게임 파이썬 구현과 관련해 어떤 도움이 필요하신가요?  
예를 들어  
  • 미로(맵) 생성 방법  
  • 최단 경로 탐색(DFS/BFS/다익스트라 등)  
  • 아바타 이동 로직  
  • GUI(그래픽) 구현  
등 구체적으로 요청해 주시면 예제 코드를 포함해 설명해 드리겠습니다.
****************************************
tokens: CompletionUsage(completion_tokens=372, prompt_tokens=49, total_tokens=421, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=256, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
CPU times: total: 62.5 ms
Wall time: 6.07 s


In [3]:
%%time

user_content = ''' 
# 명령어 설명 
- moveForward(): 앞으로 1칸 전진.
- turnLeft(): 좌측으로 90도 회전.
- turnRight(): 우측으로 90도 회전.
- turnBack(): 180도 회전.
- notDone(): 아바타가 도착지에 도착한 경우
- pathAhead(): 앞으로 이동 가능한 경우
- pathRight(): 우측으로 이동 가능한 경우
- pathLeft(): 좌측으로 이동 가능한 경우
- pathCrossroad(): 사거리인 경우
- pathAheadRight(): 우측 삼거리인 경우
- pathAheadLeft(): 좌측 삼거리인 경우
- pathRightLeft(): 갈림길인 경우
'''

messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)

******************** answer ********************
아래 코드는 “오른손 법칙”을 이용해 미로를 자동으로 탈출하는 가장 간단한 예제입니다.  
아바타가 출발점에서 출발해서, 우측 벽을 계속 따라가며(우측에 길이 없으면 직진, 직진 불가면 좌회전, 교차로ㆍ막다른 길에선 뒤돌기) 목적지에 도착할 때까지 반복합니다.

```python
def solve_maze():
    # 목적지에 도착할 때까지 반복
    while notDone():
        # 1) 우측에 길이 있으면 우회전 후 전진
        if pathRight():
            turnRight()
            moveForward()

        # 2) 우측이 막혔고, 앞이 뚫려 있으면 전진
        elif pathAhead():
            moveForward()

        # 3) 우측·앞이 모두 막혔고, 좌측에 길이 있으면 좌회전 후 전진
        elif pathLeft():
            turnLeft()
            moveForward()

        # 4) 어느 쪽도 막혀 있으면 뒤돌아서 옴
        else:
            turnBack()

    # 목적지 도착!
    print("탈출 완료!")

# 실제 호출
solve_maze()
```

설명  
  • pathRight(), pathAhead(), pathLeft() 센서를 차례로 확인하며 “우측 먼저” 우선순위로 이동  
  • 세 방향 모두 막혔을 땐 turnBack() 으로 180° 회전후 다음 루프에서 위 로직 반복  
  • notDone() 이 False(도착) 가 되면 루프 종료  

필요에 따라  
  • “왼손 법칙”을 쓰고 싶다면 pathLeft() → turnLeft() 우선순위를 앞쪽에 두시면 되고,  
  • 교차로나 삼거리(pathCrossroad, pathA

In [4]:
%%time

user_content = ''' 
# 명령의 수 세기
- 문제에는 명령의 수에 제한이 있으며, 명령어 개수를 세는 방법은 아래 규칙을 따른다.
- for, while, if, else, elif, moveForward(), turnRight(), turnLeft(), turnBack(), pathAhead(), pathRight(), pathLeft(), pathCrossroad(), pathAheadRight(), pathAheadLeft(), pathRightLeft()은  모두 각각 1번 사용할 때마다 사용한 명령어 개수가 1개씩 증가.
- 나머지 다른 모든 명령은 명령 사용 개수에 포함되지 않음.
'''

messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)

******************** answer ********************
위 예제 코드에서 다음과 같이 개수화할 수 있습니다.

1. while notDone(): → while (1)  
2. if pathRight(): → if (1) + pathRight() (1)  
3. turnRight() (1)  
4. moveForward() (1)  
5. elif pathAhead(): → elif (1) + pathAhead() (1)  
6. moveForward() (1)  
7. elif pathLeft(): → elif (1) + pathLeft() (1)  
8. turnLeft() (1)  
9. moveForward() (1)  
10. else: → else (1)  
11. turnBack() (1)  

총합 14개입니다.  
(나머지 함수 호출(print, solve_maze 함수 정의/호출 등)은 카운트에서 제외)
****************************************
tokens: CompletionUsage(completion_tokens=1049, prompt_tokens=877, total_tokens=1926, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=832, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
CPU times: total: 15.6 ms
Wall time: 14 s


In [ ]:
%%time

user_content = ''' 
<질문 프롬프트>

[s,0,1,0,0,0]
[1,0,1,1,1,1]
[1,0,1,0,1,0]
[1,1,1,0,1,0]
[0,0,0,1,1,1]
[0,0,0,g,0,0]

s가 아바타 시작위치이고  g가 goal이야
아바타는 시작 위치에서 아래쪽을 바라보고 있어
0은 길이 없는 곳이고 1은 길이 있는 곳이야
명령의 수 제한은 9개야
아바타가 g까지 갈 수 있도록 파이썬 코드를 작성해줘


(미로 배열은 따로 정의하지 않아도 된다.)
(결과는 코드만 출력)
'''

messages.append({"role": "user", "content": user_content})

response = client.chat.completions.create(
    model=model,
    messages=messages)
messages.append(response.choices[0].message)

print('*'*20, 'answer', '*'*20)
print(response.choices[0].message.content)
print('*'*40)
print('tokens:', response.usage)